# Effect of adding oil on growth of E. coli MG1655 #

### Notebook can be used to generate Fig. 4b and Supplementary Fig. S3

In [2]:
#Import necessary modules

import impact as                             impt
import impact.plotting as                    implot
from impact.parsers import Parser as parser

import pandas as pd
from openpyxl import load_workbook
from scipy.stats import linregress, ttest_ind
import numpy as np
import sys
from plotly import tools, subplots
import plotly.graph_objs as go
import pickle
import plotly.io as pio
pio.templates.default = "none"

if 'ipykernel' in sys.modules:
    from plotly.offline import init_notebook_mode
    from plotly.offline import iplot as plot
    from IPython.display import HTML
    HTML("""
         <script>
          var waitForPlotly = setInterval( function() {
          if( typeof(window.Plotly) !== "undefined" ){
          MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
          MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
          clearInterval(waitForPlotly);}}, 250 );
        </script>
        """
    )
    init_notebook_mode(connected=True)

### Update impact settings ###

In [8]:
impt.settings.perform_curve_fit = False
impt.settings.fit_type = 'gompertz'
impt.settings.outlier_cleaning_flag = False
impt.settings.max_fraction_replicates_to_remove = 0.5
impt.settings.std_deviation_cutoff=0.05
impt.settings.verbose = False
impt.settings.live_calculations = False
impt.settings.use_filtered_data = True

impt.settings.death_phase_hyperparameter = 4
impt.settings.savgolFilterWindowSize = 9

### Load Cell density data ###

In [9]:
data_format='spectromax_OD'
plate_type = '96 Wells'
file_name = 'OilTest_Anaerobic_01022021.xlsx'
expt_anaer = parser.parse_raw_data(data_format=data_format,file_name=file_name,plate_type=plate_type)
expt_anaer.calculate()

data_format='tecan'
plate_type = '96 Wells'
file_name = 'OilTest_Aerobic_01022021.xlsx'
expt_aer = parser.parse_raw_data(data_format=data_format,file_name=file_name,plate_type=plate_type)
expt_aer.calculate()


Importing data from OilTest_Anaerobic_01022021.xlsx...0.1s
Parsing time point list...Parsed 4896 time points in 0.5s
Parsing analyte list...Parsed 96 single trials in 305.0ms
Parsing single trial list...Parsed 20 replicates in 0.1s
Analyzing data...Ran analysis in 1.4s


Importing data from OilTest_Aerobic_01022021.xlsx...0.1s
Extracting timepoint data...Extracted time point data in 3.0s
Parsing time point list...Parsed 4896 time points in 0.5s
Parsing analyte list...Parsed 96 single trials in 294.3ms
Parsing single trial list...Parsed 20 replicates in 0.1s
Analyzing data...Ran analysis in 1.6s



### Plotting Function ###

In [5]:
def growth_plot(replicate_trials=None, analyte='OD600', colors=None,
                        cl_scales=['8', 'qual', 'Set1'],
                        pts_per_hour=60,plot_start=0, plot_end=-1
                        ):
    traces = []
    colors = ['rgb(54, 146, 68)',
              'rgb(217, 72, 42)',
              'rgb(33, 100, 175)',
              'rgb(146, 54, 132)',
              'rgb(175, 127, 91)']
    fillcolors = ['rgba(54, 146, 68, 0.25)',
                  'rgba(217, 72, 42, 0.25)',
                  'rgba(33, 100, 175, 0.25)',
                  'rgba(146, 54, 132, 0.25)',
                 'rgba(175, 127, 91, 0.25)']
    replicate_trials = sorted(replicate_trials, key=lambda rep: str(rep.trial_identifier))
    
    for index, replicate in enumerate(replicate_trials):

        required_num_pts = replicate.t[plot_start:plot_end][-1] * pts_per_hour
        removePointFraction = int(len(replicate.t[plot_start:plot_end]) / required_num_pts)
        if removePointFraction < 1:  removePointFraction = 1
        time = replicate.t[plot_start:plot_end]

        avg_od = replicate.avg.analyte_dict[analyte].data_vector[plot_start:plot_end]
        std_od = replicate.std.analyte_dict[analyte].data_vector[plot_start:plot_end]
        shape='linear'

        feature = 'specific_productivity'
        avg_mu = getattr(replicate.avg.analyte_dict[analyte], feature)[plot_start:plot_end]
        std_mu = getattr(replicate.std.analyte_dict[analyte], feature)[plot_start:plot_end]
        shape='spline'
            
        y = avg_od[::removePointFraction]
        y_plus = avg_od[::removePointFraction] + std_od[::removePointFraction]
        y_minus = avg_od[::removePointFraction] - std_od[::removePointFraction]
        
        mu = avg_mu[::removePointFraction]
        mu_plus = avg_mu[::removePointFraction] + std_mu[::removePointFraction]
        mu_minus = avg_mu[::removePointFraction] - std_mu[::removePointFraction]

        traces.append(go.Scatter(x=time[::removePointFraction],
                                 y=y,
                                 mode='lines',
                                 line={'color':colors[index], 'shape':shape,'width':2.5},
                                 showlegend=True, yaxis='y2',
                                 legendgroup=str(replicate.trial_identifier),
                                 name=str(replicate.trial_identifier.media)+ ", DO: "+
                                      str(replicate.trial_identifier.environment.DO)))


        traces.append(go.Scatter(x=list(time[::removePointFraction])+list(time[::removePointFraction][::-1]),
                                 y=list(y_plus) + list(y_minus[::-1]),
                                 mode='lines',yaxis='y2',
                                 line={'color':'rgba(0,0,0,0)', 'shape':shape},
                                 fill='toself',
                                 fillcolor=fillcolors[index],
                                 showlegend=False,
                                 legendgroup=str(replicate.trial_identifier),
                                 name="error_"+str(replicate.trial_identifier.media)+ ", DO: "+
                                      str(replicate.trial_identifier.environment.DO)))
        
        
        traces.append(go.Scatter(x=time[::removePointFraction],
                         y=mu,
                         mode='lines',
                         marker={'color':colors[index]},
                         line={'color':colors[index], 'shape':shape,'width':2.5, 'dash':'dot'},
                         showlegend=False, yaxis='y1',
                         legendgroup=str(replicate.trial_identifier),
                         name=str(replicate.trial_identifier.media)+ ", DO: "+
                              str(replicate.trial_identifier.environment.DO)))


        traces.append(go.Scatter(x=list(time[::removePointFraction])+list(time[::removePointFraction][::-1]),
                                 y=list(mu_plus) + list(mu_minus[::-1]),
                                 mode='lines',
                                 line={'color':'rgba(0,0,0,0)', 'shape':shape},
                                 fill='toself',
                                 fillcolor=fillcolors[index],
                                 showlegend=False, yaxis='y1',
                                 legendgroup=str(replicate.trial_identifier),
                                 name="error_"+str(replicate.trial_identifier.media)+ ", DO: "+
                                      str(replicate.trial_identifier.environment.DO)))
        
        

    return traces


### Plot growth rate and cell density for different conditions and strains ###

In [10]:
pts_per_hour = 6
feature=None

layout = go.Layout(height=425, width=650, legend_x= 1.3,

                   xaxis=dict(title='Time (h)',
                              titlefont=dict(family='Myriad Pro', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8,
                              tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(0,6)),

                   yaxis2=dict(title='Cell Density (a.u. OD600)',
                              titlefont=dict(family='Myriad Pro', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, range=(0,1.1), tickvals=(0,0.25,0.5,0.75,1,1.25),
                              tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 
                              showgrid=True,zeroline=False,anchor='x',side='right'),
                  
                  yaxis=dict(title='Instantaneous Growth Rate (1/h)',
                            titlefont=dict(family='Myriad Pro', size=18, color='black'),

                              anchor='x', side='left', showgrid=True, zeroline=False,
                            tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                              range=(0, 2.2),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, overlaying='y2'))


rep_list = [replicate for replicate in expt_aer.replicate_trials if
            str(replicate.trial_identifier.strain.parent) == 'WT' and  
            list(replicate.trial_identifier.media.components.values())[0].concentration!=200 and
            'HeavyOil' not in str(replicate.trial_identifier.media.components)]

rep_list = rep_list + [replicate for replicate in expt_anaer.replicate_trials if
            str(replicate.trial_identifier.strain.parent) == 'WT' and 
                      'Vol' not in str(replicate.trial_identifier.media.components) and
                      'HeavyOil' not in str(replicate.trial_identifier.media.components)]

rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier))

tracelist = growth_plot(replicate_trials=rep_list,
                                pts_per_hour=pts_per_hour,
                                plot_start=0, plot_end=37)
if tracelist:
    fig = go.Figure(data=tracelist, layout=layout)
    plot(fig)#, image='svg')
    #pio.write_image(fig,"Figures/fig_3_growthcurve.svg",format='svg')


In [11]:
pts_per_hour = 6
feature=None
layout = go.Layout(height=425, width=650, legend=dict(x= 1.3),

                   xaxis=dict(title='Time (h)',
                              titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8,
                              tickfont=dict(size=16, family='Arial', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(0,8)),

                   yaxis2=dict(title='Cell Density (a.u. OD600)',
                              titlefont=dict(family='Myriad Pro regular', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, range=(0,1.2),
                              tickfont=dict(family='Myriad Pro regular',size=16, color='black'), tickcolor='black', 
                              showgrid=True,zeroline=False,anchor='x',side='left'),
                  
                  yaxis=dict(title='Instantaneous Growth Rate (1/h)',
                            titlefont=dict(family='Myriad Pro', size=18, color='black'),
                             visible=False,

                              anchor='x', side='right', showgrid=False, zeroline=False,
                            tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                              range=(0, 2.2),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0))

rep_list = [replicate for replicate in expt_anaer.replicate_trials if
            str(replicate.trial_identifier.strain.parent) == 'WT' and 
                      list(replicate.trial_identifier.media.components.values())[0].concentration!=200 and
                      'HeavyOil' not in str(replicate.trial_identifier.media.components)]

rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier))

tracelist = growth_plot(replicate_trials=rep_list,
                                pts_per_hour=pts_per_hour,
                                plot_start=0, plot_end=51)
if tracelist:
    fig = go.Figure(data=tracelist, layout=layout)
    fig.update()
    plot(fig)#, image='svg')
    #pio.write_image(fig,"Figures/si_fig_2.svg",format='svg')


In [16]:
pts_per_hour = 9
feature=None

layout = go.Layout(height=425, width=650, legend_x= 1.3,

                   xaxis=dict(title='Time (h)',
                              titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8,
                              tickfont=dict(size=16, family='Arial', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(0,8)),

                   yaxis2=dict(title='Cell Density (a.u. OD600)',
                              titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, range=(0,1.1), tickvals=(0,0.25,0.5,0.75,1,1.25),
                              tickfont=dict(family='Arial',size=16, color='black'), tickcolor='black', 
                              showgrid=True,zeroline=False,anchor='x',side='right'),
                  
                  yaxis=dict(title='Instantaneous Growth Rate (1/h)',
                            titlefont=dict(family='Arial', size=18, color='black'),

                              anchor='x', side='left', showgrid=True, zeroline=False,
                            tickfont=dict(family='Arial',size=16, color='black'), tickcolor='black', 

                              range=(0, 2.2),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, overlaying='y2'))


rep_list = [replicate for replicate in expt_aer.replicate_trials if
            str(replicate.trial_identifier.strain.parent) == 'Δ(adh,pta)-D1' and  
            list(replicate.trial_identifier.media.components.values())[0].concentration!=200 and
            'HeavyOil' not in str(replicate.trial_identifier.media.components)]

rep_list = rep_list + [replicate for replicate in expt_anaer.replicate_trials if
            str(replicate.trial_identifier.strain.parent) == 'Δ(adh,pta)-D1' and 
                      'Vol' not in str(replicate.trial_identifier.media.components) and
                      'HeavyOil' not in str(replicate.trial_identifier.media.components)]

rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier))

tracelist = growth_plot(replicate_trials=rep_list,
                                pts_per_hour=pts_per_hour,
                                plot_start=0, plot_end=51)
if tracelist:
    fig = go.Figure(data=tracelist, layout=layout)
    plot(fig)
    #pio.write_image(fig,"Figures/si_fig_3_d1_1.svg",format='svg')

    
feature=None
layout = go.Layout(height=425, width=650, legend=dict(x= 1.3),

                   xaxis=dict(title='Time (h)',
                              titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8,
                              tickfont=dict(size=16, family='Arial', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(0,8)),

                   yaxis2=dict(title='Cell Density (a.u. OD600)',
                              titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, range=(0,1.2),
                              tickfont=dict(family='Arial',size=16, color='black'), tickcolor='black', 
                              showgrid=True,zeroline=False,anchor='x',side='left'),
                  
                  yaxis=dict(title='Instantaneous Growth Rate (1/h)',
                            titlefont=dict(family='Arial', size=18, color='black'),
                             visible=False,
                             anchor='x', side='right', showgrid=False, zeroline=False,
                             tickfont=dict(family='Arial',size=16, color='black'), tickcolor='black', 
                              range=(0, 2.2),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0))

rep_list = [replicate for replicate in expt_anaer.replicate_trials if
            str(replicate.trial_identifier.strain.parent) == 'Δ(adh,pta)-D1' and 
                      list(replicate.trial_identifier.media.components.values())[0].concentration!=200 and
                      'HeavyOil' not in str(replicate.trial_identifier.media.components)]

rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier))

tracelist = growth_plot(replicate_trials=rep_list,
                                pts_per_hour=pts_per_hour,
                                plot_start=0, plot_end=51)
if tracelist:
    fig = go.Figure(data=tracelist, layout=layout)
    fig.update()
    plot(fig)
    #pio.write_image(fig,"Figures/si_fig_3_d1_2.svg",format='svg')


In [17]:
pts_per_hour = 9
feature=None

layout = go.Layout(height=425, width=650, legend_x= 1.3,

                   xaxis=dict(title='Time (h)',
                              titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8,
                              tickfont=dict(size=16, family='Arial', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(0,8)),

                   yaxis2=dict(title='Cell Density (a.u. OD600)',
                              titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, range=(0,1.1), tickvals=(0,0.25,0.5,0.75,1,1.25),
                              tickfont=dict(family='Arial',size=16, color='black'), tickcolor='black', 
                              showgrid=True,zeroline=False,anchor='x',side='right'),
                  
                  yaxis=dict(title='Instantaneous Growth Rate (1/h)',
                            titlefont=dict(family='Arial', size=18, color='black'),

                              anchor='x', side='left', showgrid=True, zeroline=False,
                            tickfont=dict(family='Arial',size=16, color='black'), tickcolor='black', 

                              range=(0, 2.2),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, overlaying='y2'))


rep_list = [replicate for replicate in expt_aer.replicate_trials if
            str(replicate.trial_identifier.strain.parent) == 'Δ(adh,pta)-D28' and  
            list(replicate.trial_identifier.media.components.values())[0].concentration!=200 and
            'HeavyOil' not in str(replicate.trial_identifier.media.components)]

rep_list = rep_list + [replicate for replicate in expt_anaer.replicate_trials if
            str(replicate.trial_identifier.strain.parent) == 'Δ(adh,pta)-D28' and 
                      'Vol' not in str(replicate.trial_identifier.media.components) and
                      'HeavyOil' not in str(replicate.trial_identifier.media.components)]

rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier))

tracelist = growth_plot(replicate_trials=rep_list,
                                pts_per_hour=pts_per_hour,
                                plot_start=0, plot_end=51)
if tracelist:
    fig = go.Figure(data=tracelist, layout=layout)
    plot(fig)
    #pio.write_image(fig,"Figures/si_fig_3_d28_1.svg",format='svg')

    
feature=None
layout = go.Layout(height=425, width=650, legend=dict(x= 1.3),

                   xaxis=dict(title='Time (h)',
                              titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8,
                              tickfont=dict(size=16, family='Arial', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(0,8)),

                   yaxis2=dict(title='Cell Density (a.u. OD600)',
                              titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, range=(0,1.2),
                              tickfont=dict(family='Arial',size=16, color='black'), tickcolor='black', 
                              showgrid=True,zeroline=False,anchor='x',side='left'),
                  
                  yaxis=dict(title='Instantaneous Growth Rate (1/h)',
                            titlefont=dict(family='Arial', size=18, color='black'),
                             visible=False,

                              anchor='x', side='right', showgrid=False, zeroline=False,
                            tickfont=dict(family='Arial',size=16, color='black'), tickcolor='black', 

                              range=(0, 2.2),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0))

rep_list = [replicate for replicate in expt_anaer.replicate_trials if
            str(replicate.trial_identifier.strain.parent) == 'Δ(adh,pta)-D28' and 
                      list(replicate.trial_identifier.media.components.values())[0].concentration!=200 and
                      'HeavyOil' not in str(replicate.trial_identifier.media.components)]

rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier))

tracelist = growth_plot(replicate_trials=rep_list,
                                pts_per_hour=pts_per_hour,
                                plot_start=0, plot_end=51)
if tracelist:
    fig = go.Figure(data=tracelist, layout=layout)
    fig.update()
    plot(fig)
    #pio.write_image(fig,"Figures/si_fig_3_d28_2.svg",format='svg')


In [18]:
pts_per_hour = 9
feature=None

layout = go.Layout(height=425, width=650, legend_x= 1.3,

                   xaxis=dict(title='Time (h)',
                              titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8,
                              tickfont=dict(size=16, family='Arial', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(0,8)),

                   yaxis2=dict(title='Cell Density (a.u. OD600)',
                              titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, range=(0,1.1), tickvals=(0,0.25,0.5,0.75,1,1.25),
                              tickfont=dict(family='Arial',size=16, color='black'), tickcolor='black', 
                              showgrid=True,zeroline=False,anchor='x',side='right'),
                  
                  yaxis=dict(title='Instantaneous Growth Rate (1/h)',
                            titlefont=dict(family='Arial', size=18, color='black'),

                              anchor='x', side='left', showgrid=True, zeroline=False,
                            tickfont=dict(family='Arial',size=16, color='black'), tickcolor='black', 

                              range=(0, 2.2),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, overlaying='y2'))


rep_list = [replicate for replicate in expt_aer.replicate_trials if
            str(replicate.trial_identifier.strain.parent) == 'Δ(adh,pta)-D59' and  
            list(replicate.trial_identifier.media.components.values())[0].concentration!=200 and
            'HeavyOil' not in str(replicate.trial_identifier.media.components)]

rep_list = rep_list + [replicate for replicate in expt_anaer.replicate_trials if
            str(replicate.trial_identifier.strain.parent) == 'Δ(adh,pta)-D59' and 
                      'Vol' not in str(replicate.trial_identifier.media.components) and
                      'HeavyOil' not in str(replicate.trial_identifier.media.components)]

rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier))

tracelist = growth_plot(replicate_trials=rep_list,
                                pts_per_hour=pts_per_hour,
                                plot_start=0, plot_end=51)
if tracelist:
    fig = go.Figure(data=tracelist, layout=layout)
    plot(fig)#, image='svg')
    #pio.write_image(fig,"Figures/si_fig_3_d59_1.svg",format='svg')

    
feature=None
layout = go.Layout(height=425, width=650, legend=dict(x= 1.3),

                   xaxis=dict(title='Time (h)',
                              titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8,
                              tickfont=dict(size=16, family='Arial', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(0,8)),

                   yaxis2=dict(title='Cell Density (a.u. OD600)',
                              titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, range=(0,1.2),
                              tickfont=dict(family='Arial',size=16, color='black'), tickcolor='black', 
                              showgrid=True,zeroline=False,anchor='x',side='left'),
                  
                  yaxis=dict(title='Instantaneous Growth Rate (1/h)',
                            titlefont=dict(family='Arial', size=18, color='black'),
                             visible=False,

                              anchor='x', side='right', showgrid=False, zeroline=False,
                            tickfont=dict(family='Arial',size=16, color='black'), tickcolor='black', 

                              range=(0, 2.2),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0))

rep_list = [replicate for replicate in expt_anaer.replicate_trials if
            str(replicate.trial_identifier.strain.parent) == 'Δ(adh,pta)-D59' and 
                      list(replicate.trial_identifier.media.components.values())[0].concentration!=200 and
                      'HeavyOil' not in str(replicate.trial_identifier.media.components)]

rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier))

tracelist = growth_plot(replicate_trials=rep_list,
                                pts_per_hour=pts_per_hour,
                                plot_start=0, plot_end=51)
if tracelist:
    fig = go.Figure(data=tracelist, layout=layout)
    fig.update()
    plot(fig)#, image='svg')
    #pio.write_image(fig,"Figures/si_fig_3_d59_2.svg",format='svg')
